## Searching for binding in Nasmyth 1 Axis

The Nasmyth 1 axis has observed multiple events where the torque limit is exceeded. This is believed to be a binding inside the rotator. On 2019-11-19, multiple tests were carried out to try to reproduce the binding. Although unsuccessful, this notebook looks at some of the data taken during this time in hopes to see small irregularities.

In this notebook we use [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](summit-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [ ]:
client = aioinflux.InfluxDBClient(host='summit-influxdb-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.  Specify a time range for data in `InfluxQL`.  The default is 20hrs ago, but this may need to be changed depending on how recently data was taken.

In [ ]:
client.output = 'dataframe'
#time_span = "time >= '2019-09-08T01:41:00Z' AND time <= '2019-09-08T01:44:00Z'"
#time_span = "time >= '2019-09-18T00:47:00Z' AND time <= '2019-09-19T04:49:00Z'"
time_span = "time >= '2019-09-20T04:00:00Z' AND time <= '2019-09-20T04:04:00Z'"
time_span = "time >= '2019-09-20T03:55:00Z' AND time <= '2019-09-20T04:02:00Z'"

# stopped around Tue Nov 19 00:17:12 UTC 2019
time_span = "time >= '2019-11-18T23:00:00Z' AND time <= '2019-11-19T00:17:00Z'"
time_span = "time >= '2019-11-18T23:07:00Z' AND time <= '2019-11-18T23:16:00Z'"

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
async def get_data_frame(field_base, topic, els=100):
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

In [ ]:
nas1_position_measured = await get_data_frame('nasmyth1CalculatedAngle', 'lsst.sal.ATMCS.mount_Nasmyth_Encoders')
#az1 = await get_data_frame('azimuthEncoder1Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
# az2 = await get_data_frame('azimuthEncoder2Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
# az3 = await get_data_frame('azimuthEncoder3Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
#az_raw = await client.query(f'SELECT "azimuthCalculatedAngle99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.mount_AzEl_Encoders" WHERE {time_span}')

In [ ]:
nas1_position_demand = await get_data_frame('nasmyth1RotatorAngle', 'lsst.sal.ATMCS.trajectory')

In [ ]:
nas1_position_measured

In [ ]:
# Looks like demanded Torques are too small to get measured...
nas1_motor_torque_demand = await get_data_frame('nasmyth1MotorTorque', 'lsst.sal.ATMCS.torqueDemand')
nas1_motor_torque_measured = await get_data_frame('nasmyth1MotorTorque', 'lsst.sal.ATMCS.measuredTorque')

In [ ]:
nas1_measured_vel = await get_data_frame('elevationMotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
nas1_demanded_vel = await get_data_frame('nasmyth1RotatorAngleVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
p = figure(x_axis_type='datetime', y_range=(-30,180 ), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth1 (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=nas1_position_demand.index.values, y=nas1_position_demand['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='Commanded Nasymth1 Position')
p.line(x=nas1_position_measured.index.values, y=nas1_position_measured['nasmyth1CalculatedAngle'], color='lightblue', line_width=2, legend_label='Computed Nasmyth1 Position')
#p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend='ATPng Target El')

p.extra_y_ranges = {'Velocity': Range1d(start=-2.5, end=2.5)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.cross(x=nas1_demanded_vel.index.values, y=nas1_demanded_vel['nasmyth1RotatorAngleVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='Measured Nasmyth1 Motor Velocity')
p.line(x=nas1_measured_vel.index.values, y=nas1_measured_vel['elevationMotorVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='Commanded Nasmyth1 Motor Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
p = figure(x_axis_type='datetime', y_range=(-30,180 ), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Nasmyth1 (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=nas1_position_demand.index.values, y=nas1_position_demand['nasmyth1RotatorAngle'], color='black', line_width=2, legend_label='Commanded Nasymth1 Position')
p.line(x=nas1_position_measured.index.values, y=nas1_position_measured['nasmyth1CalculatedAngle'], color='lightblue', line_width=2, legend_label='Computed Nasmyth1 Position')
#p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend='ATPng Target El')

p.extra_y_ranges = {'Torque': Range1d(start=-2.0, end=2.0)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque'), 'right')
p.cross(x=nas1_motor_torque_demand.index.values, y=nas1_motor_torque_demand['nasmyth1MotorTorque'], color='red', alpha=0.5, y_range_name='Torque', legend_label='Measured Nasmyth1 Motor Torque')
p.line(x=nas1_motor_torque_measured.index.values, y=nas1_motor_torque_measured['nasmyth1MotorTorque'], color='black', alpha=0.5, y_range_name='Torque', legend_label='Commanded Nasmyth1 Motor Torque')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)